## 1. 试分别用 (1) Jacobi 迭代 (2) Gauss-Seidel 迭代法 和（3）共轭梯度法, 解线性方程组
$$\begin{bmatrix}
 10 & 1 & 2 & 3 & 4 \\
 1 &9 &-1& 2&-3\\
 2&-1 &7&3 & -5\\
 3 &2 &3& 12&-1\\
 4 &-3 &-5& -1&15
\end{bmatrix}\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4\\
x_5
\end{bmatrix}=\begin{bmatrix}
12\\
-27\\
14\\
-17\\
12
\end{bmatrix}
$$

## 迭代初始向量为$x^{(0)}=\begin{bmatrix}0&0&0&0&0\end{bmatrix}^T$

## 解：三对角矩阵使用追赶法,进行LU分解，再求解方程

### Code

In [1]:
using LinearAlgebra
A=[10 1 2 3 4;
 1 9 -1 2 -3;
 2 -1 7 3 -5;
 3 2 3 12 -1;
 4 -3 -5 -1 15] 
b=[12;-27;14;-17;12]
x0=[0;0;0;0;0]
ϵ=0.5*1e-3
iter_num=100

100

## 将迭代式写成矩阵形式 $x^{k+1}=Mx^{k}+g$
## 为叙述方便，将A按下三角L，对角D，上三角U划分，记A=D-L-U
### (1)对于Jacobbi 迭代法，$M=I-D^{-1}A,g=D^{-1}b$

In [2]:
function Jacobi(A,b,x0,ϵ,iter_num)
    D=diagm(0=>diag(A,0)*1.0)
    E=Matrix{Float64}(I,size(A))#cannot assign variable LinearAlgebra.I
    M=I-D^(-1)*A
    g=D^(-1)*b
    xn=copy(x0)
    for i=1:iter_num
        xn_plus = M*xn+g
        if abs(norm(xn_plus-xn)<ϵ)
            println("Jacobbi iter success")
            return xn_plus,norm(xn_plus-xn),i
        else
            xn=xn_plus
        end
    end
    
    if abs(norm(xn_plus-xn)<ϵ)
        println("Jacobbi iter success")
    else
        println("Jacobbi iter failed")
    end
    return xn_plus,norm(xn_plus-xn),iter_num
end

Jacobi (generic function with 1 method)

### 根据给定条件，解得方程组的解如下

In [3]:
xn,error,iters=Jacobi(A,b,x0,ϵ,iter_num)

Jacobbi iter success


([1.00086, -2.00079, 2.99855, -1.99977, 0.998963], 0.0004505829050280891, 34)

In [4]:
xn

5-element Array{Float64,1}:
  1.0008644520246608
 -2.00079168104775  
  2.998548689104864 
 -1.99976665890548  
  0.9989626473538343

### (2)对于G-S迭代法，$x^{k+1}=D^{-1}Lx^{k+1}+D^{-1}Ux^{k}+D^{-1}b$，所以
### 有，$x^{k+1}=(I-D^{-1}L)^{-1}D^{-1}Ux^{k}+(I-D^{-1}L)^{-1}D^{-1}b$，即
### $x^{k+1}=(D-L)^{-1}Ux^{k}+(D-L)^{-1}b=Mx^k+g$

In [5]:
function GS(A,b,x0,ϵ,iter_num)
    D=diagm(0=>diag(A,0)*1.0)
    L=LowerTriangular(A)*(-1.0)
    U=UpperTriangular(A)*(-1.0)
    M=(D-L)^(-1)*U
    g=(D-L)^(-1)*b
    xn=copy(x0)
    for i=1:iter_num
        xn_plus = M*xn+g
        if abs(norm(xn_plus-xn)<ϵ)
            println("G-S iter success")
            return xn_plus,norm(xn_plus-xn),i
        else
            xn=xn_plus
        end
    end
    
    if abs(norm(xn_plus-xn)<ϵ)
        println("G-S iter success")
    else
        println("G-S iter failed")
    end
    return xn_plus,norm(xn_plus-xn),iter_num
end

GS (generic function with 1 method)

In [6]:
#?lower
#?upper

### 根据给定条件，解得方程组的解如下

In [7]:
xn,error,iters=GS(A,b,x0,ϵ,iter_num)

G-S iter success


([0.40232, -0.924464, 0.713279, -0.507254, 0.237266], 0.00030360437006398945, 14)

In [8]:
xn

5-element Array{Float64,1}:
  0.40231976771448696
 -0.9244644888196436 
  0.7132793128226385 
 -0.5072535379408231 
  0.23726587972501256

### (3)共轭梯度法将求解变成求优化问题的极值，沿着关于A共轭的 $d$ 进行搜索，而非基于计算误差得到的负梯度$r$。具体原理此处不表。

In [9]:
#求解
function ConjGrad(A,b,x0,ϵ,iter_num)
    dn=b-A*x0
    rn=copy(dn)
    λn = rn'dn/(dn'A*dn)
    xn=x0+λn*dn
    
    for i=1:iter_num
        rn = b-A*xn
        if abs(norm(rn)<ϵ)
            println("Conjugate Gradient iter success")
            return xn,norm(rn),i
        else
            β=-rn'A*dn/(dn'A*dn)
            dn=rn+ β*dn
            λn = rn'dn/(dn'A*dn)
            xn=xn+λn*dn
        end
    end
    
    if abs(norm(rn)<ϵ)
        println("Conjugate Gradient iter success")
    else
        println("Conjugate Gradient iter failed")
    end
    return xn,norm(rn),iter_num
end

ConjGrad (generic function with 1 method)

### 根据给定条件，解得方程组的解如下

In [10]:
xn,error,iters=ConjGrad(A,b,x0,ϵ,iter_num)

Conjugate Gradient iter success


([1.0, -2.0, 3.0, -2.0, 1.0], 1.5280799322856756e-14, 5)

In [11]:
xn

5-element Array{Float64,1}:
  1.0000000000000004
 -2.0               
  3.0000000000000004
 -1.9999999999999996
  0.9999999999999997